In [1]:
import re
import json
import glob
import datetime
from pathlib import Path

In [2]:
now = datetime.datetime.now()
time = now.strftime("%Y-%m-%dT%H:%M:%S")
time

'2022-08-20T13:31:56'

In [3]:
SIZE = (1024, 1024)
LABELS_DIR = r'runs/detect/instances_train2017_yolov5/exp_test_mine_lr0=0_005_lrf=0_05_momentum=0_997_Adam_2022-07-24_23:03:25/wsi/debug_conf001_iou45/labels'

In [12]:
coco_json = {
    "info": {
        "year": "2022",
        "version": "1.0",
        "description": "Cell object detection predictions on WSI testis data",
        "contributor": "BIIT research group at University of Tartu",
#         "url": "",
        "date_created": time
    },
    "licenses": [],
    "categories": [
        {
            "id": 1,
            "name": "Spermatogonia",
            "supercategory": "cell"
        },
        {
            "id": 2,
            "name": "Sertoli",
            "supercategory": "cell"
        },
        {
            "id": 3,
            "name": "Primary spermatocyte",
            "supercategory": "cell"
        },
        {
            "id": 4,
            "name": "Spermatid",
            "supercategory": "cell"
        },
        {
            "id": 5,
            "name": "Spermatozoa",
            "supercategory": "cell"
        },
#         {
#             "id": 6,
#             "name": "Garbage cell",
#             "supercategory": "cell"
#         }
    ],
    "images": [],
    "annotations": []
}

In [13]:
Path(LABELS_DIR)

PosixPath('runs/detect/instances_train2017_yolov5/exp_test_mine_lr0=0_005_lrf=0_05_momentum=0_997_Adam_2022-07-24_23:03:25/wsi/debug_conf001_iou45/labels')

In [16]:
wsi_cache = {} 
img_cnt = 0
ann_cnt = 0

for p in Path(LABELS_DIR).glob('*.txt'):
    print(p)
    stem = p.stem

    wsix = int(re.findall(r'x=(\d+)', stem)[1])
    wsiy = int(re.findall(r'y=(\d+)', stem)[0])
    wsiname = re.findall(r'(.*)_x=', stem)[0] + '.mrxs'
    
    img_id = wsi_cache.get(wsiname)
    if img_id is None:
        img_id = img_cnt
        wsi_cache[wsiname] = img_id
        
        wsi = {
#             "coco_url": ""
            "id": img_id,
#             "license": 1,
            "file_name": wsiname,
            "width": SIZE[0],
            "height": SIZE[1],
#             "date_captured": "",
#             "flickr_url": ""
        } 
        img_cnt += 1
    
        coco_json['images'].append(wsi)
    
    content = p.read_text().strip()
    for line in content.split('\n'):
        cid, cx, cy, cw, ch, cc_ = list(map(float, line.split()))
        
        bbox = SIZE[0] * (cx - cw / 2) + wsix, SIZE[1] * (cy - ch / 2) + wsiy, SIZE[0] * cw, SIZE[1] * ch
        bbox = list(map(lambda x: int(round(x)), bbox))
        area = bbox[2] * bbox[3]
        ann_id = ann_cnt
        
        annotation = {
            "id": ann_id,
            "image_id": img_id,
            "category_id": cid,
            "bbox": bbox,
            "area": area,
            "iscrowd": 0,
#             "segmentation": []
        }
        ann_cnt += 1
        
        coco_json["annotations"].append(annotation)

runs/detect/instances_train2017_yolov5/exp_test_mine_lr0=0_005_lrf=0_05_momentum=0_997_Adam_2022-07-24_23:03:25/wsi/debug_conf001_iou45/labels/19H11010_0111_x=109568_y=197632.txt


IndexError: list index out of range

In [7]:
# coco_json

In [13]:
with open('19H11010_0111.json', 'w') as f:
    json.dump(coco_json, f)

In [23]:
p.parents[1] / 'wsi'

PosixPath('runs/detect/instances_train2017_yolov5/exp_test_mine_lr0=0_005_lrf=0_05_momentum=0_997_Adam_2022-07-24_23:03:25/wsi/debug_conf001_iou45/wsi')